In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np


In [1]:
def getTestandTrainSplit():
    iris_df_train =  pd.read_csv(r'C:\Users\rajes\OneDrive\Documents\AMPBA\DL\iris_training.csv', header=None)
    iris_df_test =  pd.read_csv(r'C:\Users\rajes\OneDrive\Documents\AMPBA\DL\iris_test.csv', header=None)

    x_train = iris_df_train.iloc[:,0:4].values
    y_train = iris_df_train.iloc[:,-1].values

    x_test = iris_df_test.iloc[:,0:4].values
    y_test = iris_df_test.iloc[:,-1].values
    
    return x_train,y_train, x_test,y_test


In [4]:
x_train,y_train, x_test,y_test = getTestandTrainSplit()

In [8]:
# SETTING UP HYPERPARAMETERS AND DATA SET PARAMETERS
num_classes = 3 # -- Iris Setosa -- Iris Versicolour -- Iris Virginica
num_features = 4 # 1. sepal length in cm 2. sepal width in cm  3. petal length in cm 4. petal width in cm

# Training parameters.
learning_rate = 0.001
training_steps = 5000
batch_size = 10
display_step = 100
hidden1=5
hidden2=10
hidden3=5


# LOADING AND PREPARING THE IRIS DATA SET
x_train,y_train, x_test,y_test = getTestandTrainSplit()

# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Normalize images value from [0, 255] to [0, 1].
#x_train, x_test = x_train / 255., x_test / 255.

print(len(x_train),len(x_test),len(y_train),len(y_test))

# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Use tf.data API to shuffle and batch data.
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

#prefetch: Creates a `Dataset` that prefetches elements from this dataset. 
#Like other `Dataset` methods, prefetch operates on the #elements of the input dataset. 
#It has no concept of examples vs. batches. `examples.prefetch(2)` will prefetch 
#two elements (2 examples), #while `examples.batch(20).prefetch(2)` will prefetch 
#2 elements (2 batches, of 20 examples each).
train_data=train_data.shuffle(x_train.shape[0]).repeat().batch(batch_size).prefetch(1)

120 30 120 30


In [11]:
import math
# INITIALIZING WEIGHTS AND BIASES
W1 = tf.Variable(tf.random.truncated_normal([num_features, hidden1],
                            stddev=1.0 / math.sqrt(float(num_features))), name='weights1')
b1 = tf.Variable(tf.zeros([hidden1]), name="bias1")


W2 = tf.Variable(tf.random.truncated_normal([hidden1, hidden2],
                            stddev=1.0 / math.sqrt(float(hidden1))), name='weights2')
b2 = tf.Variable(tf.zeros([hidden2]), name="bias2")


W3 = tf.Variable(tf.random.truncated_normal([hidden2, hidden3],
                            stddev=1.0 / math.sqrt(float(hidden2))), name='weights3')
b3 = tf.Variable(tf.zeros([hidden3]), name="bias3")


W4 = tf.Variable(tf.random.truncated_normal([hidden3, num_classes],
                            stddev=1.0 / math.sqrt(float(hidden3))), name='weights4')
b4 = tf.Variable(tf.zeros([num_classes]), name="bias4")



# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# DEFINING OPTIMIZERS AND ACCURACY METRICS
# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of the highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# DEFINING LOGISTIC REGRESSION AND COST FUNCTION
# Logistic regression (Wx + b).
def mlp(x):
    o1=tf.nn.relu(tf.matmul(x, W1) + b1)
    o2=tf.nn.relu(tf.matmul(o1, W2) + b2)
    o3=tf.nn.relu(tf.matmul(o2, W3) + b3)
    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.matmul(o3, W4) + b4)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

# OPTIMIZATION PROCESS AND UPDATING WEIGHTS AND BIASES
def run_optimization(x, y):
# Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = mlp(x)
        loss = cross_entropy(pred, y)
    # Compute gradients.
    gradients = g.gradient(loss, [W1, b1, W2, b2,W3, b3,W3, b4])
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W1, b1, W2, b2,W3, b3, W3,b4]))
    
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = mlp(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))
    if step % 1000 ==0:
        print("Test Accuracy: %f" % accuracy(mlp(x_test), y_test))
        
# Test model on test set.
pred = mlp(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

step: 100, loss: 10.820010, accuracy: 0.400000
step: 200, loss: 9.287359, accuracy: 0.800000
step: 300, loss: 8.237388, accuracy: 0.800000
step: 400, loss: 7.017076, accuracy: 0.800000
step: 500, loss: 8.824913, accuracy: 0.300000
step: 600, loss: 7.086841, accuracy: 0.500000
step: 700, loss: 7.574911, accuracy: 0.300000
step: 800, loss: 6.591817, accuracy: 0.600000
step: 900, loss: 6.852276, accuracy: 0.400000
step: 1000, loss: 6.667493, accuracy: 0.600000
Test Accuracy: 0.700000
step: 1100, loss: 4.380307, accuracy: 0.800000
step: 1200, loss: 5.024455, accuracy: 0.500000
step: 1300, loss: 6.185556, accuracy: 0.800000
step: 1400, loss: 5.382369, accuracy: 0.800000
step: 1500, loss: 5.711068, accuracy: 0.900000
step: 1600, loss: 3.679421, accuracy: 1.000000
step: 1700, loss: 2.648060, accuracy: 0.900000
step: 1800, loss: 5.725039, accuracy: 0.800000
step: 1900, loss: 3.835852, accuracy: 0.900000
step: 2000, loss: 4.599768, accuracy: 0.800000
Test Accuracy: 0.900000
step: 2100, loss: 3.